In [1]:
import os
import json
import wandb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')
# Set seed
np.random.seed(42)

# set font to 12 and times new roman

plt.rcParams.update({'font.size': 12})
plt.rcParams["font.family"] = "Times New Roman"

from utils import *






The `LightGBM` module could not be imported. To enable LightGBM support in Darts, follow the detailed instructions in the installation guide: https://github.com/unit8co/darts/blob/master/INSTALL.md
The `Prophet` module could not be imported. To enable Prophet support in Darts, follow the detailed instructions in the installation guide: https://github.com/unit8co/darts/blob/master/INSTALL.md
The `CatBoost` module could not be imported. To enable CatBoost support in Darts, follow the detailed instructions in the installation guide: https://github.com/unit8co/darts/blob/master/INSTALL.md


In [2]:

os.chdir(r"..") # should be the git repo root directory, checking below:
print("Current working directory: " + os.getcwd())
assert os.getcwd()[-8:] == "WattCast"

Current working directory: /Users/nikolaushouben/Desktop/WattCast


In [3]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nikolaushouben (wattcast). Use `wandb login --relogin` to force relogin


True

In [4]:
run = wandb.init(project='Wattcast', name='results_synthesis', id = 'results_synthesis', resume=True)

project_name = "Wattcast"
# Initialize your project
api = wandb.Api()
runs = api.runs(project_name)

## Results imports


### Grabbing all metrics from all runs from wandb

In [5]:

def get_run_name_id_dict(runs):
    '''Loops through all runs and returns a dictionary of run names and ids'''
    name_id_dict = {}
    for run_ in runs:
        l = run_.name.split('_')[:-1]
        l.insert(2, 'in')
        n = '_'.join(l)
        # remove the _ around in
        n = n.replace('_in_', 'in')
        name_id_dict[n] = run_.id
    return name_id_dict


name_id_dict = get_run_name_id_dict(runs)

In [6]:
name_id_dict

{'results_in': 'results_synthesis',
 '3_villageinvillage_2': '3_village_village_2_60min',
 '3_villageinvillage_1': '3_village_village_1_60min',
 '3_villageinvillage_0': '3_village_village_0_60min',
 '2_townintown_2': '2_town_town_2_60min',
 '2_townintown_1': '2_town_town_1_60min',
 '2_townintown_0': '2_town_town_0_60min',
 '1_countyinSacramento': '1_county_Sacramento_60min',
 '1_countyinNew_York': '1_county_New_York_60min',
 '1_countyinLos_Angeles': '1_county_Los_Angeles_60min',
 '4_neighborhoodingermany': '4_neighborhood_germany_60min'}

# Paper Figures & Tables

### Mappings for plotting shapes and colors

In [7]:
model2shape = {
    'RandomForest': 'o',
    'LightGBMModel': 's',
    'XGBModel': 'D',
    'BlockRNNModel': 'x',
    'NBEATSModel': '^',
    'TFTModel': 'v',
    'LinearRegressionModel': '*',
    '48-Hour Persistence': 'P'
}


model2color = {
    'RandomForest': 'blue',
    'LightGBMModel': 'orange',
    'XGBModel': 'green',
    'BlockRNNModel': 'red',
    'NBEATSModel': 'purple',
    'TFTModel': 'pink',
    'LinearRegressionModel': 'brown',
    '48-Hour Persistence': 'black'
}


shape2model = {v: k for k, v in model2shape.items()}


metric_dict = {'mape': 'Mean Absolute Percentage Error (MAPE)',
               'rmse': 'Root Mean Squared Error (RMSE)',
                'mae': 'Mean Absolute Error (MAE)',
                'smape': 'Symmetric Mean Absolute Percentage Error (SMAPE)',
                'rmse_skill_score': 'RMSE Skill Score',
                'r2_score': 'R2 Score',
                'max_peak_error': 'Max Peak Error',
                'mean_n_peak_error': 'Mean N Peak Error',
                }

season2color = {'Summer': 'orange', 'Winter': 'blue'}
color2season = {v: k for k, v in season2color.items()}

horizon2color = {'Ground Truth': 'black',
                '1 Hours Ahead': 'blue',
                '4 Hours Ahead': 'green',
                '8 Hours Ahead': 'orange',
                '24 Hours Ahead': 'red',
                '48 Hours Ahead': 'purple'}


model_groups = {'Tree-based': ['RandomForest', 'LightGBMModel', 'XGBModel'],
                         'Neural Network': ['BlockRNNModel', 'NBEATSModel', 'TFTModel'],}

model2group = {}
for group, models in model_groups.items():
    for model in models:
        model2group[model] = group


group2shape = {'Tree-based': 'x', 'Neural Network': 'o'}
shape2group = {v: k for k, v in group2shape.items()}

group2color = {'Tree-based': 'blue', 'Neural Network': 'red'}
color2group = {v: k for k, v in group2color.items()}


scale2unit = {'county': 'GW', 'town': 'MW', 'village': 'kW', 'neighborhood': 'kW'}
unit2kWh = {'W':1/1000, 'kW': 1, 'MW': 1000, 'GW': 1000000}


index2county_locations = {0: 'Los_Angeles', 1: 'New_York', 2: 'Sacramento'}
index2town_locations = {0: 'town_0', 1: 'town_1', 2: 'town_2'}
index2village_locations = {0: 'village_0', 1: 'village_1', 2: 'village_2'}
index2neighborhood_locations = {0: 'neighborhood_0', 1: 'neighborhood_1', 2: 'neighborhood_2'}
scale2ordering = {'county': index2county_locations, 'town': index2town_locations, 'village': index2village_locations, 'neighborhood': index2neighborhood_locations}






In [ ]:
# importing error metric tables (used in most sections below)
key_list = list(name_id_dict.keys())
for key in key_list:
    if key[0] not in ['1', '2', '3', '4', '5']:
        del name_id_dict[key]
        
dfs_sorted = {}
for name, id in name_id_dict.items():
    try:
        run_path = f'wattcast/{project_name}/run-{id}-Errormetrics:latest'
        artifact = run.use_artifact(f'{run_path}', type='run_table')
        artifact_dir = artifact.download()
        with open(os.path.join(artifact_dir, os.listdir(artifact_dir)[0])) as f:
            data = json.load(f)
        df_metrics = pd.DataFrame(data['data'], columns=data['columns'])
        df = df_metrics.sort_values(by=['model', 'season'])
        dfs_sorted[name] = df
    except:
        print(f'Error in {name}')


## Per Model

### Plot 1: Skill Score vs Horizon for each model and season

In [ ]:
# Set these:

spatial_scale = 'town'
metric_of_interest = 'rmse_skill_score'


dfs_sorted_plot = dfs_sorted.copy()
keys = list(dfs_sorted_plot.keys())
for key in keys:
    if not spatial_scale in key:
        del dfs_sorted_plot[key]



fig, axs = plt.subplots(1, len(dfs_sorted_plot), figsize=(5*(len(dfs_sorted_plot)),5), sharex=True, sharey=True)

    
axs = axs.ravel()

# Create a list to store the handles and labels for the legend
legend_handles = []
legend_labels = []

for i, ax in enumerate(axs):
    # Map the colors and shapes to the DataFrame
    name, df = list(dfs_sorted_plot.items())[i]
    df['color'] = df['season'].map(season2color)
    df['shape'] = df['model'].map(model2shape)

    # exclude 24 hour persistence
    df = df.loc[ df['model'] != '48-Hour Persistence']
    
    for shape in set(df['shape']):
        for color in set(df.loc[df['shape'] == shape, 'color']):
            x = df.loc[(df['shape'] == shape) & (df['color'] == color), 'horizon_in_hours']
            y = df.loc[(df['shape'] == shape) & (df['color'] == color), metric_of_interest]
            label = f'{shape2model[shape]} ({color2season[color]})'
            scatter = ax.scatter(x, y, c=color, marker=shape, label=label)
            if label not in legend_labels:
                legend_handles.append(scatter)
                legend_labels.append(label)

    # Set the x ticks and labels
    ax.set_xticks([1, 4, 8, 24, 48])
    ax.set_xlabel('Forecast Horizon (hours)')

    # Set the y label
    if i == 0:
        ax.set_ylabel(f'{metric_dict[metric_of_interest]}')

    # Set the title for each subplot
    ax.set_title(f'{name.split("in")[-1].replace("_", " ")}')

    # Add grid for each subplot
    ax.grid(True)

    # Add a horizontal red dashed line at y = 0
    if metric_of_interest == 'rmse_skill_score':
        ax.axhline(y=0, color='red', linestyle='--')

# Create a single legend for all subplots
fig.legend(handles=legend_handles, labels=legend_labels, title='Model', loc='lower center', bbox_to_anchor=(0.5, -0.2), ncol=5, frameon=True)

# Adjust the spacing between subplots
fig.tight_layout()

# Show the plot
plt.show()


In [ ]:
for format in ['png', 'pdf']:
    fig.savefig(os.path.join(os.getcwd(),'imgs','figures',f'{spatial_scale}{metric_of_interest}.{format}'), bbox_inches='tight')

### Plot 2: Side by side comparison of models for each season for a selected week

In [ ]:
# set these
run_to_visualize = '4_neighborhoodingermany'
season = 'Summer'
algorithm = 'XGBModel'


scale = run_to_visualize.split('_')[1].split('in')[0]
unit = scale2unit[scale]
conversion = unit2kWh[unit]

In [ ]:
files = get_file_names(project_name, name_id_dict, run_to_visualize, season)

side_by_side_plots_dict= download_plotly_plots(get_latest_plotly_plots(files))

df_all = side_by_side_df(side_by_side_plots_dict)

df_all

### Plot

In [ ]:
# find the day with the highest peak
dt_highest_peak = pd.Timestamp(df_all['Ground Truth'].idxmax()).date()
dt_start = dt_highest_peak - pd.Timedelta(days=2)
dt_end = dt_highest_peak + pd.Timedelta(days=2)

df_all.index = pd.to_datetime(df_all.index)
df_all = df_all.loc[dt_start:dt_end]



fig, ax = plt.subplots(figsize=(12,5))

df_preds = df_all.filter(like=algorithm)
df_preds.columns = [col.split(':')[-1][1:] + ' Ahead' for col in df_preds.columns]
df_gt = df_all.filter(like='Ground')
df_plot = pd.concat([df_gt, df_preds], axis=1)
for col in df_plot.columns:
    ax.plot(df_plot[col], label=col, color=horizon2color[col], linestyle= 'solid' if col == 'Ground Truth' else ':')

ax.set_ylabel(f'Electricity Load ({unit})')
ax.set_title(f'Ground Truth and {algorithm} Predictions for {run_to_visualize}')

ax.grid(True, axis='x')
ax.legend(frameon=True)

In [ ]:
for format in ['png', 'pdf']:
    fig.savefig(os.path.join(os.getcwd(),'imgs','figures',f'plot_3_side-by-side_horizons_heat_wave_{run_to_visualize}_{season}_{algorithm}.{format}'), bbox_inches='tight')

#wandb.log({f'plot_3_side-by-side_horizons_heat_wave_{run_to_visualize}_{season}_{algorithm}': wandb.Image(fig)})

### Plot 3: Comparing Spatial Scales (mean of seasons and datasets)

In [ ]:
metric_of_interest = 'rmse'

df_metrics_all_scales = pd.DataFrame()

for scale, df in dfs_sorted.items():
    scale = scale.split('in')[0]
    df['scale'] = scale
    df_metrics_all_scales = df_metrics_all_scales.append(df)

# marginalize over the season
df_metrics_grouped = df_metrics_all_scales.groupby(['scale', 'model', 'horizon_in_hours']).mean()[[metric_of_interest]].reset_index()

# extract the model with the best performance for each scale and horizon
df_metrics_grouped = df_metrics_grouped.sort_values(by=['scale', metric_of_interest], ascending=False if metric_of_interest == 'rmse_skill_score' else True)
df_metrics_grouped = df_metrics_grouped.drop_duplicates(subset=['scale', 'horizon_in_hours'], keep='first')
df_metrics_grouped = df_metrics_grouped.sort_values(by=['scale', 'horizon_in_hours'])
df_metrics_grouped = df_metrics_grouped.reset_index(drop=True)


# Define colors and shapes based on the season and model
df = df_metrics_grouped
fig, ax = plt.subplots()

# Create a list to store the handles and labels for the legend
legend_handles = []
legend_labels = []


# Map the colors and shapes to the DataFrame
df['shape'] = df['model'].map(model2shape)


for scale, df_scale in df.groupby('scale'): # this for loop is to make sure the order of scales is from county to neighborhood
    for shape in set(df_scale['shape']):
        x = df_scale.loc[(df_scale['shape'] == shape), 'horizon_in_hours']
        y = df_scale.loc[(df_scale['shape'] == shape), 'scale']
        label = f'{shape2model[shape]}'
        scatter = ax.scatter(x, y, marker=shape, label=label, color=model2color[shape2model[shape]])
        if label not in legend_labels:
            legend_handles.append(scatter)
            legend_labels.append(label)

# Set the x ticks and labels
ax.set_xticks([1, 4, 8, 24, 48])
ax.set_xlabel('Forecast Horizon (hours)')


ax.set_ylabel('Spatial Scale')

# Set the title for each subplot
fig.suptitle(f'Best Model by {metric_dict[metric_of_interest]}')

# Create a single legend for all subplots
fig.legend(handles=legend_handles, labels=legend_labels, title='Model', loc='lower center', bbox_to_anchor=(0.6, -0.25), ncol=2, frameon=True)


# Show the plot
plt.show()


In [ ]:

for format in ['png', 'pdf']:
    fig.savefig(os.path.join(os.getcwd(),'imgs','figures',f'plot_1_overview_across_scales_{metric_of_interest}.{format}'), bbox_inches='tight')

#wandb.log({f'plot_1_overview_across_scales_{metric_of_interest}': fig})

### Plot 4: Comparing Spatial Scales (mean of datasets per season)

In [ ]:
df_metrics_all_scales = pd.DataFrame()

for scale, df in dfs_sorted.items():
    scale = scale.split('in')[0]
    df['scale'] = scale
    df_metrics_all_scales = df_metrics_all_scales.append(df)

df_metrics_grouped = df_metrics_all_scales.groupby(['season', 'scale', 'model', 'horizon_in_hours']).mean()[[metric_of_interest]].reset_index()

season = 'Winter'

fig, axs = plt.subplots(1,2, figsize=(12,5), sharex=True, sharey=True)

axs = axs.ravel()
# Create a list to store the handles and labels for the legend
legend_handles = []
legend_labels = []

for i, ax in enumerate(axs):

    season = list(season2color.keys())[i]

    df_metrics_grouped_season = df_metrics_grouped.loc[df_metrics_grouped['season'] == season]

    df = get_best_model_per_scale_and_horizon(df_metrics_grouped_season, metric_of_interest)

    shape2model = {v: k for k, v in model2shape.items()}

    # Map the colors and shapes to the DataFrame
    df['shape'] = df['model'].map(model2shape)

    for scale, df_scale in df.groupby('scale'):
        for shape in set(df_scale['shape']):
            x = df_scale.loc[(df_scale['shape'] == shape), 'horizon_in_hours']
            y = df_scale.loc[(df_scale['shape'] == shape), 'scale']
            label = f'{shape2model[shape]}'
            scatter = ax.scatter(x, y, marker=shape, label=label, color=model2color[shape2model[shape]])
            if label not in legend_labels:
                legend_handles.append(scatter)
                legend_labels.append(label)

    # Set the x ticks and labels
    ax.set_xticks([1, 4, 8, 24, 48])
    ax.set_xlabel('Forecast Horizon (hours)')
    
    #ax.set_yticklabels(sorted(ax.get_yticklabels(), key=lambda x: x.get_text()))


    # Set the y label
    if i == 0:
        ax.set_ylabel('Spatial Scale')

    # Set the title for each subplot
    ax.set_title(f'{season}')


# Create a single legend for all subplots
fig.legend(handles=legend_handles, labels=legend_labels, title='Model', loc='lower center', bbox_to_anchor=(0.5, -0.25), ncol=2, frameon=True)

fig.suptitle(f'Best Model by {metric_dict[metric_of_interest]}')
# Adjust the spacing between subplots
#fig.tight_layout()

# Show the plot
plt.show()


## Tree-based models vs. Deep learning models

### Plot 1

In [ ]:
# Set these:

spatial_scale = 'village'
metric_of_interest = 'rmse_skill_score'

dfs_sorted_plot = dfs_sorted.copy()
keys = list(dfs_sorted_plot.keys())
for key in keys:
    if not spatial_scale in key:
        del dfs_sorted_plot[key]


# Define colors and shapes based on the season and model
color2season = {v: k for k, v in season2color.items()}
shape2model = {v: k for k, v in model2shape.items()}


fig, axs = plt.subplots(1, len(dfs_sorted_plot), figsize=(5*(len(dfs_sorted_plot)),5), sharex=True, sharey=True)
axs = axs.ravel()

# Create a list to store the handles and labels for the legend
legend_handles = []
legend_labels = []

for i, ax in enumerate(axs):
    # Map the colors and shapes to the DataFrame
    name, df = list(dfs_sorted_plot.items())[i]
    # exclude 24 hour persistence
    df = df.loc[df['model'] != '48-Hour Persistence']
    df['group'] = df['model'].map(model2group)
    df = df.groupby(['group', 'horizon_in_hours', 'season']).mean().reset_index()
    df['color'] = df['season'].map(season2color)
    df['shape'] = df['group'].map(group2shape)
    
    
    for shape in set(df['shape']):
        for color in set(df.loc[df['shape'] == shape, 'color']):
            x = df.loc[(df['shape'] == shape) & (df['color'] == color), 'horizon_in_hours']
            y = df.loc[(df['shape'] == shape) & (df['color'] == color), metric_of_interest]
            label = f'{shape2group[shape]} ({color2season[color]})'
            scatter = ax.scatter(x, y, c=color, marker=shape, label=label)
            if label not in legend_labels:
                legend_handles.append(scatter)
                legend_labels.append(label)

    # Set the x ticks and labels
    ax.set_xticks([1, 4, 8, 24, 48])
    ax.set_xlabel('Forecast Horizon (hours)')

    # Set the y label
    if i == 0:
        ax.set_ylabel(f'{metric_dict[metric_of_interest]}')

    # Set the title for each subplot
    ax.set_title(f'{name.split("in")[-1].replace("_", " ")}')

    # Add grid for each subplot
    ax.grid(True)

    # Add a horizontal red dashed line at y = 0
    if metric_of_interest == 'rmse_skill_score':
        ax.axhline(y=0, color='red', linestyle='--')

# Create a single legend for all subplots
fig.legend(handles=legend_handles, labels=legend_labels, title='Model', loc='lower center', bbox_to_anchor=(0.5, -0.2), ncol=5, frameon=True)

# Adjust the spacing between subplots
fig.tight_layout()

# Show the plot
plt.show()


### Plot 3

In [ ]:
metric_of_interest = 'mape'

df_metrics_all_scales = pd.DataFrame()

for scale, df in dfs_sorted.items():
    scale = scale.split('in')[0]
    df['scale'] = scale
    df_metrics_all_scales = df_metrics_all_scales.append(df)

# marginalize over the season
df_metrics_grouped = df_metrics_all_scales.groupby(['scale', 'model', 'horizon_in_hours']).mean()[[metric_of_interest]].reset_index()

# extract the model with the best performance for each scale and horizon
df_metrics_grouped = df_metrics_grouped.sort_values(by=['scale', metric_of_interest], ascending=False if metric_of_interest == 'rmse_skill_score' else True)
df_metrics_grouped = df_metrics_grouped.drop_duplicates(subset=['scale', 'horizon_in_hours'], keep='first')
df_metrics_grouped = df_metrics_grouped.sort_values(by=['scale', 'horizon_in_hours'])
df_metrics_grouped = df_metrics_grouped.reset_index(drop=True)


# Define colors and shapes based on the season and model
df = df_metrics_grouped
shape2model = {v: k for k, v in model2shape.items()}

fig, ax = plt.subplots()

# Create a list to store the handles and labels for the legend
legend_handles = []
legend_labels = []


# Map the colors and shapes to the DataFrame
df['group'] = df['model'].map(model2group)
df = df.groupby(['group', 'horizon_in_hours', 'scale']).mean().reset_index()

df['shape'] = df['group'].map(group2shape)

for scale, df_scale in df.groupby('scale'): # this for loop is to make sure the order of scales is from county to neighborhood
    for shape in set(df_scale['shape']):
        x = df_scale.loc[(df_scale['shape'] == shape), 'horizon_in_hours']
        y = df_scale.loc[(df_scale['shape'] == shape), 'scale']
        label = f'{shape2group[shape]}'
        scatter = ax.scatter(x, y, marker=shape, label=label, color=group2color[shape2group[shape]])
        if label not in legend_labels:
            legend_handles.append(scatter)
            legend_labels.append(label)

# Set the x ticks and labels
ax.set_xticks([1, 4, 8, 24, 48])
ax.set_xlabel('Forecast Horizon (hours)')

ax.set_ylabel('Spatial Scale')

# Set the title for each subplot
fig.suptitle(f'Best Model by {metric_dict[metric_of_interest]}')

# Create a single legend for all subplots
fig.legend(handles=legend_handles, labels=legend_labels, title='Model', loc='lower center', bbox_to_anchor=(0.6, -0.25), ncol=2, frameon=True)


# Show the plot
plt.show()

for format in ['png', 'pdf']:
    fig.savefig(os.path.join(os.getcwd(),'imgs','figures',f'plot_1_overview_across_scales_{metric_of_interest}.{format}'), bbox_inches='tight')

#wandb.log({f'plot_1_overview_across_scales_{metric_of_interest}': fig})

## MPC Evaluation

In [8]:
name_id_dict

{'results_in': 'results_synthesis',
 '3_villageinvillage_2': '3_village_village_2_60min',
 '3_villageinvillage_1': '3_village_village_1_60min',
 '3_villageinvillage_0': '3_village_village_0_60min',
 '2_townintown_2': '2_town_town_2_60min',
 '2_townintown_1': '2_town_town_1_60min',
 '2_townintown_0': '2_town_town_0_60min',
 '1_countyinSacramento': '1_county_Sacramento_60min',
 '1_countyinNew_York': '1_county_New_York_60min',
 '1_countyinLos_Angeles': '1_county_Los_Angeles_60min',
 '4_neighborhoodingermany': '4_neighborhood_germany_60min'}

In [9]:
# set these
run_to_visualize = list(name_id_dict.keys())[-1]
season = 'Summer'

scale = run_to_visualize.split('_')[1].split('in')[0]
unit = scale2unit[scale]
conversion = unit2kWh[unit]

In [10]:
files = get_file_names(project_name, name_id_dict, run_to_visualize, season)

side_by_side_plots_dict= download_plotly_plots(get_latest_plotly_plots(files))

df_all = side_by_side_df(side_by_side_plots_dict)

df_all *= conversion

In [11]:
df_all = df_all.iloc[:,-3:]

In [16]:
from pyomo.environ import (
    ConcreteModel,
    Set,
    Param,
    Var,
    Constraint,
    Objective,
    Reals,  # type: ignore
    NonNegativeReals,  # type: ignore
)
from pyomo.opt import SolverFactory
import pandas as pd
from datetime import timedelta
import os
import re
import argparse
import wandb
from utils import infer_frequency
import numpy as np


def get_forecasts(df, h, fc_type, horizon):
    df = df.loc[(df.index > h) & (df.index < (h + timedelta(hours=horizon + 1)))]
    fct = df[fc_type].to_list()
    return fct


def construct_fc_types(df_fc):
    fc_types = {}
    for col in df_fc.iloc[:, :-1].columns:
        horizon = re.findall(r"\d+", col)[0]
        fc_types[col] = int(horizon)

    fc_types[df_fc.columns[-1]] = max(list(fc_types.values()))
    return fc_types


def run_opt(
    load_forecast,
    prices,
    bss_energy,
    bss_duration,
    bss_eff,
    bss_size,
    monthly_peak,
    tier_load_magnitude,
    tier2_multiplier,
    peak_cost,
):
    ################################
    # MPC optimization model
    ################################
    m = ConcreteModel()
    m.T = Set(initialize=list(range(len(load_forecast))))

    # Params
    m.energy_prices = Param(m.T, initialize=prices)
    m.demand = Param(m.T, initialize=load_forecast)
    m.bss_size = Param(initialize=bss_size)
    m.bss_eff = Param(initialize=bss_eff)
    m.bss_energy_start = Param(initialize=bss_energy)
    m.bss_max_pow = Param(initialize=bss_size / bss_duration)
    m.monthly_peak = Param(initialize=monthly_peak)
    m.tier_load_magnitude = Param(initialize=tier_load_magnitude)
    m.tier2_multiplier = Param(initialize=tier2_multiplier)
    m.peak_cost = Param(initialize=peak_cost)

    # variables
    m.net_load = Var(m.T, domain=Reals)
    m.tier1_net_load = Var(m.T, domain=Reals)
    m.tier2_net_load = Var(m.T, domain=NonNegativeReals)
    m.peak = Var(domain=NonNegativeReals)
    m.dev_peak_plus = Var(domain=NonNegativeReals)
    m.dev_peak_minus = Var(domain=NonNegativeReals)
    m.bss_p_ch = Var(m.T, domain=Reals)
    m.bss_en = Var(m.T, domain=NonNegativeReals)

    def energy_balance(m, t):
        return m.net_load[t] == m.bss_eff * m.bss_p_ch[t] + m.demand[t]

    m.energy_balance = Constraint(m.T, rule=energy_balance)

    def tier_load_definition(m, t):
        return m.net_load[t] == m.tier1_net_load[t] + m.tier2_net_load[t]

    m.tier_load_definition = Constraint(m.T, rule=tier_load_definition)

    def tier_load_limit(m, t):
        return m.tier1_net_load[t] <= m.tier_load_magnitude

    m.tier_load_limit = Constraint(m.T, rule=tier_load_limit)

    def operation_peak(m, t):
        return m.peak >= m.net_load[t]

    m.operation_peak = Constraint(m.T, rule=operation_peak)

    def relevant_peak(m):
        return m.peak - m.monthly_peak == m.dev_peak_plus - m.dev_peak_minus

    m.relevant_peak = Constraint(rule=relevant_peak)

    def bat_soc(m, t):
        if t == 0:
            return m.bss_en[t] == m.bss_energy_start + m.bss_p_ch[t]
        else:
            return m.bss_en[t] == m.bss_en[t - 1] + m.bss_p_ch[t]

    m.bat_soc = Constraint(m.T, rule=bat_soc)

    def bat_lim_energy(m, t):
        return m.bss_en[t] <= m.bss_size

    m.bat_lim_energy = Constraint(m.T, rule=bat_lim_energy)

    def bat_lim_power_pos(m, t):
        return m.bss_p_ch[t] <= m.bss_max_pow

    m.bat_lim_power_pos = Constraint(m.T, rule=bat_lim_power_pos)

    def bat_lim_power_neg(m, t):
        return -m.bss_max_pow <= m.bss_p_ch[t]

    m.bat_lim_power_neg = Constraint(m.T, rule=bat_lim_power_neg)

    def cost(m):
        return (
            sum(m.tier1_net_load[t] * m.energy_prices[t] for t in m.T)
            + sum(
                m.tier2_net_load[t] * m.energy_prices[t] * m.tier2_multiplier
                for t in m.T
            )
            + (m.dev_peak_plus + m.monthly_peak) * m.peak_cost
        )

    m.ObjectiveFunction = Objective(rule=cost)

    opt = SolverFactory("cplex")
    results = opt.solve(m)

    # get results
    res_df = pd.DataFrame(index=list(m.T))
    for v in [m.net_load, m.tier1_net_load, m.tier2_net_load, m.bss_p_ch, m.bss_en]:
        res_df = res_df.join(pd.Series(v.get_values(), name=v.getname()))

    # just returns the first (next time) set-point
    sp = res_df.iloc[0].to_dict()

    return sp


def run_operations(
    hours_of_simulation,
    fc,
    fc_type,
    prices,
    horizon,
    bat_size_kwh,
    bat_duration,
    bss_eff,
    initial_soc,
    tier_load_magnitude,
    tier2_multiplier,
    peak_cost,
):
    # function to run operations of given a forecast and system data

    # initializing monthly peak (it will store the historical peak)
    peak = tier_load_magnitude  # initializing monthly peak

    # initialize energy in the battery with the initial soc
    energy_in_the_battery = bat_size_kwh * initial_soc

    operations = {}
    for t in fc.index[:hours_of_simulation]:
        # get load forecast as a list
        load = get_forecasts(df=fc, h=t, fc_type=fc_type, horizon=horizon)

        # get the energy prices as a list
        ep = get_forecasts(
            df=prices, h=t, fc_type="ep", horizon=horizon
        )

        # gets a set_point for the next interval based on the MPC optimization
        # the horizon of the optimization is given by the length of the forecast
        set_point = run_opt(
            load_forecast=load,
            prices=ep,
            bss_energy=energy_in_the_battery,
            bss_size=bat_size_kwh,
            bss_duration=bat_duration,
            monthly_peak=peak,
            tier_load_magnitude=tier_load_magnitude,
            tier2_multiplier=tier2_multiplier,
            peak_cost=peak_cost,
            bss_eff=bss_eff,
        )

        # implement set point in time, calculate net and tier load
        set_point_time = t + timedelta(hours=1)
        net_load = fc.loc[set_point_time]["Ground Truth"] + set_point["bss_p_ch"]
        tier1_load = (
            net_load if net_load <= tier_load_magnitude else tier_load_magnitude
        )
        tier2_load = (
            0 if net_load <= tier_load_magnitude else net_load - tier_load_magnitude
        )

        # implement peak condition
        if peak < net_load:
            peak = net_load
        set_point.update(
            {
                "opr_net_load": net_load,
                "opr_tier1_load": tier1_load,
                "opr_tier2_load": tier2_load,
            }
        )

        # update energy in the battery
        energy_in_the_battery = set_point["bss_en"]

        operations.update({set_point_time: set_point})

    return pd.DataFrame(operations).T


def scale_by_gt(df):
    """Scale the predictions and ground truth by the max and min of the ground truth"""

    gt_max = df["Ground Truth"].max()
    gt_min = df["Ground Truth"].min()

    df_scaled = df.copy()
    df_scaled["Ground Truth"] = (df_scaled["Ground Truth"] - gt_min) / (gt_max - gt_min)

    for col in df_scaled.columns:
        if col != "Ground Truth":
            df_scaled[col] = (df_scaled[col] - gt_min) / (gt_max - gt_min)

    return df_scaled, gt_max, gt_min


def generate_ep_profile(df, hour_shift=1, mu=0.05, sigma=0.1):
    """Generate electricity price profiles based on the ground truth of the load"""

    timesteps_per_hour = int(infer_frequency(df) // 60)
    shift_in_timesteps = hour_shift * timesteps_per_hour
    # step 1: shift the ground truth by n hours
    ep1 = df["Ground Truth"].shift(shift_in_timesteps)

    # step 2: add a random noise to it
    ep2 = ep1 + np.random.normal(mu, sigma, len(ep1))
    # step 3: smooth it
    ep3 = ep2.ewm(span=timesteps_per_hour * 6).mean().fillna(method="bfill")
    # step 4: adjust to be between roughly 0 and 0.3
    ep4 = ep3.to_frame("ep") / 3
    return ep4


def run_mpc(df_fc):
    ##############################################
    # input parameters
    #############################################

    hours_of_simulation = 600

    timesteps_per_hour = infer_frequency(df_fc)
    df_scaled, gt_max, gt_min = scale_by_gt(df_fc)

    # battery parameters
    initial_soc = 0.5  # initial state of charge of the battery (no unit)
    bat_size_kwh = 2 * timesteps_per_hour  # size of the battery in kWh
    c_rate = 0.5  # C-rate of the battery
    bat_duration = (
        c_rate * bat_size_kwh
    )  # battery duration (Max_kW = bat_size/duration)
    bat_efficiency = 0.95  # charging and discharging efficiency of the battery

    # electricity price parameters
    tier_limit = df_scaled["Ground Truth"].quantile(0.95)
    ep = generate_ep_profile(df=df_scaled)
    tier2_cost_multiplier = 1.5  # cost multiplication of the tier 2 load
    cost_of_peak = 5.0  # cost of the monthly peak

    fc_types = construct_fc_types(df_fc=df_fc)
    ##############################################
    # simulation starts here
    ##############################################
    results = pd.DataFrame()

    for fc_type, horizon in fc_types.items():
        print(f"running {fc_type}")

        fc_result = run_operations(
            hours_of_simulation=hours_of_simulation,
            fc=df_scaled,  # forecast table
            fc_type=fc_type,  # label of forecast type
            prices=ep,  # electricity prices
            horizon=horizon,
            bat_size_kwh=bat_size_kwh,
            bat_duration=bat_duration,
            initial_soc=initial_soc,
            bss_eff=bat_efficiency,
            tier_load_magnitude=tier_limit,
            tier2_multiplier=tier2_cost_multiplier,
            peak_cost=cost_of_peak,
        )

        # add the forecast label to the forecast operation results
        for k in fc_result:
            fc_result[k + f"_{fc_type}"] = fc_result[k]  # type: ignore
            fc_result = fc_result.drop(k, axis=1)

        # build the operation results table
        if results.shape[0] == 0:
            results = pd.concat([results, fc_result])
        else:
            results = results.join(fc_result)

    # calculate operation costs of each forecast
    cost_results = {}
    results = results.join(df_scaled[["Ground Truth"]]).join(ep)
    for fc_type in fc_types.keys():
        tier1 = (
            results[f"opr_tier1_load_{fc_type}"] * results["ep"]
        ).sum()

        tier2 = (
            results[f"opr_tier2_load_{fc_type}"]
            * results["ep"]
            * tier2_cost_multiplier
        ).sum()

        peak = results[f"opr_net_load_{fc_type}"].max() * cost_of_peak

        cost_results.update(
            {
                fc_type: {
                    "horizon": fc_types[fc_type],
                    "tier1": tier1,
                    "tier2": tier2,
                    "peak": peak,
                    "total": tier1 + tier2 + peak,
                }
            }
        )
    cost_results = pd.DataFrame(cost_results).T

    # write results, TODO: replace with wandb logging and return results
    cost_results.to_csv("data/results/cost_results.csv")
    results.to_csv("data/results/operation_results.csv")



In [17]:
run_mpc(df_all)

running NBEATSModel Summer - Horizon: 8 Hours


TypeError: unsupported operand type(s) for +: 'float' and 'NoneType'